# 🎯 Model Training: Validación Rigurosa SIN Data Leakage

**Objetivo:** Entrenar modelos con validación cruzada y métricas reales

**Mejoras:**
- ✅ Features corregidas (sin data leakage)
- ✅ Cross-validation (StratifiedKFold)
- ✅ Confusion matrices
- ✅ Feature importance
- ✅ Learning curves
- ✅ ROC curves (multiclass)
- ✅ Comparación antes/después

**Autor:** Asier Ortiz García  
**Fecha:** Noviembre 2025

## 📦 Imports y Configuración

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import json
import pickle
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, learning_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    f1_score, precision_score, recall_score, roc_auc_score, roc_curve
)
from sklearn.preprocessing import label_binarize
from imblearn.over_sampling import SMOTE

plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 6)

Path('../models/crohn').mkdir(parents=True, exist_ok=True)
Path('../models/cu').mkdir(parents=True, exist_ok=True)
Path('../docs/figures').mkdir(parents=True, exist_ok=True)

print("=" * 80)
print("MODEL TRAINING: Cross-Validation + Diagnósticos")
print("=" * 80)

## 🔧 Funciones de Visualización

In [ ]:
def plot_confusion_matrix(y_true, y_pred, title, save_path=None):
    """Plot confusion matrix."""
    cm = confusion_matrix(y_true, y_pred, labels=['low', 'medium', 'high'])
    
    fig, ax = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Low', 'Medium', 'High'],
                yticklabels=['Low', 'Medium', 'High'],
                ax=ax, cbar_kws={'label': 'Count'})
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=12)
    
    # Add percentages
    total = cm.sum()
    for i in range(3):
        for j in range(3):
            pct = cm[i, j] / total * 100
            ax.text(j+0.5, i+0.7, f'({pct:.1f}%)', 
                   ha='center', va='center', fontsize=9, color='gray')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_feature_importance(model, feature_names, title, save_path=None, top_n=20):
    """Plot feature importance."""
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1][:top_n]
    
    fig, ax = plt.subplots(figsize=(10, 8))
    ax.barh(range(top_n), importances[indices], color='steelblue', edgecolor='black')
    ax.set_yticks(range(top_n))
    ax.set_yticklabels([feature_names[i] for i in indices])
    ax.invert_yaxis()
    ax.set_xlabel('Importance', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.grid(True, alpha=0.3, axis='x')
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_learning_curves(model, X, y, title, save_path=None):
    """Plot learning curves to detect overfitting."""
    train_sizes, train_scores, val_scores = learning_curve(
        model, X, y, cv=5, n_jobs=-1,
        train_sizes=np.linspace(0.1, 1.0, 10),
        scoring='accuracy', random_state=42
    )
    
    train_mean = train_scores.mean(axis=1)
    train_std = train_scores.std(axis=1)
    val_mean = val_scores.mean(axis=1)
    val_std = val_scores.std(axis=1)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.plot(train_sizes, train_mean, 'o-', color='blue', label='Training score')
    ax.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1, color='blue')
    ax.plot(train_sizes, val_mean, 'o-', color='red', label='Cross-validation score')
    ax.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1, color='red')
    
    ax.set_xlabel('Training Size', fontsize=12)
    ax.set_ylabel('Accuracy', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='best')
    ax.grid(True, alpha=0.3)
    
    # Add gap indicator
    gap = train_mean[-1] - val_mean[-1]
    ax.text(0.5, 0.05, f'Train-Val Gap: {gap:.3f}', 
            transform=ax.transAxes, fontsize=11,
            bbox=dict(boxstyle='round', facecolor='yellow' if gap > 0.1 else 'lightgreen', alpha=0.5))
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

def plot_roc_curves(y_true, y_pred_proba, title, save_path=None):
    """Plot ROC curves for multiclass (one-vs-rest)."""
    classes = ['low', 'medium', 'high']
    y_true_bin = label_binarize(y_true, classes=classes)
    
    fig, ax = plt.subplots(figsize=(10, 8))
    
    for i, class_name in enumerate(classes):
        fpr, tpr, _ = roc_curve(y_true_bin[:, i], y_pred_proba[:, i])
        auc = roc_auc_score(y_true_bin[:, i], y_pred_proba[:, i])
        ax.plot(fpr, tpr, label=f'{class_name.capitalize()} (AUC = {auc:.3f})', linewidth=2)
    
    ax.plot([0, 1], [0, 1], 'k--', label='Random (AUC = 0.500)')
    ax.set_xlabel('False Positive Rate', fontsize=12)
    ax.set_ylabel('True Positive Rate', fontsize=12)
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.legend(loc='lower right')
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()

print("✓ Funciones de visualización definidas")

## 🔧 Función de Entrenamiento con Validación

In [ ]:
def train_with_cv(ibd_type='crohn'):
    """
    Entrena modelos con cross-validation y genera diagnósticos completos.
    """
    print(f"\n{'='*80}")
    print(f"ENTRENANDO: {ibd_type.upper()} (CON CROSS-VALIDATION)")
    print(f"{'='*80}\n")
    
    # Cargar dataset
    df = pd.read_csv(f'../data/processed/{ibd_type}/ml_dataset_enhanced.csv')
    clusters_df = pd.read_csv(f'../data/processed/{ibd_type}/user_clusters.csv', index_col=0)
    
    df = df.merge(clusters_df[['cluster']], left_on='user_id', right_index=True, how='left')
    
    print(f"✓ Dataset cargado: {len(df):,} registros")
    print(f"  Usuarios: {df['user_id'].nunique():,}")
    print(f"  Distribución de risk: {df['risk_level'].value_counts().to_dict()}")
    
    # Features (ELIMINANDO las que causan data leakage)
    exclude_cols = [
        'user_id', 'checkin_date', 'risk_level', 'severity_score', 'cluster',
        'sex', 'first_checkin', 'days_since_first_checkin', 'is_flare_day',
        'cumulative_flare_days', 'risk_numeric',
        # Eliminadas por data leakage:
        'total_symptom_score', 'gi_score', 'systemic_score', 'symptom_severity_category',
        'symptom_volatility_7d', 'symptom_change_rate', 'is_bad_day', 'days_since_low_symptoms',
        'vulnerable_state', 'gi_dominant'
    ]
    
    feature_cols = [col for col in df.columns if col not in exclude_cols]
    X = df[feature_cols].copy()
    y = df['risk_level'].copy()
    
    # Encode categorical
    if 'gender' in X.columns:
        X = pd.get_dummies(X, columns=['gender'], drop_first=True)
    
    # Fill missing
    for col in X.columns:
        if X[col].dtype in ['float64', 'int64']:
            X[col].fillna(X[col].median(), inplace=True)
    
    print(f"\n✅ Features limpias (sin data leakage): {len(X.columns)}")
    print(f"  Features: {list(X.columns)}")
    
    # Split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # SMOTE
    print(f"\n🔄 Aplicando SMOTE...")
    smote = SMOTE(sampling_strategy='not majority', random_state=42)
    X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
    print(f"  Antes: {len(X_train):,} | Después: {len(X_train_res):,}")
    
    # 1. CROSS-VALIDATION
    print(f"\n1️⃣  CROSS-VALIDATION (5-fold StratifiedKFold)...")
    rf_model = RandomForestClassifier(
        n_estimators=200, max_depth=15, min_samples_split=10,
        random_state=42, n_jobs=-1
    )
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    cv_scores = cross_val_score(rf_model, X_train_res, y_train_res, cv=cv, scoring='accuracy', n_jobs=-1)
    
    print(f"\n  CV Accuracy: {cv_scores.mean():.3f} (+/- {cv_scores.std():.3f})")
    print(f"  CV Scores: {[f'{s:.3f}' for s in cv_scores]}")
    
    # 2. TRAIN FINAL MODEL
    print(f"\n2️⃣  Entrenando modelo final...")
    rf_model.fit(X_train_res, y_train_res)
    
    # 3. EVALUATE
    y_pred = rf_model.predict(X_test)
    y_pred_proba = rf_model.predict_proba(X_test)
    
    test_acc = accuracy_score(y_test, y_pred)
    test_f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f"\n✅ MÉTRICAS DE TEST:")
    print(f"  Accuracy: {test_acc:.3f}")
    print(f"  F1-score (weighted): {test_f1:.3f}")
    print(f"  Gap (CV vs Test): {abs(cv_scores.mean() - test_acc):.3f}")
    
    if abs(cv_scores.mean() - test_acc) > 0.1:
        print(f"  ⚠️  WARNING: Gap > 0.1 indica posible overfitting!")
    else:
        print(f"  ✅ Gap < 0.1: Modelo generaliza bien")
    
    print(f"\n📊 Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))
    
    # 4. GRÁFICOS DE DIAGNÓSTICO
    print(f"\n4️⃣  Generando gráficos de diagnóstico...")
    
    # Confusion Matrix
    plot_confusion_matrix(
        y_test, y_pred,
        f'Confusion Matrix - {ibd_type.upper()}',
        f'../docs/figures/{ibd_type}_confusion_matrix.png'
    )
    
    # Feature Importance
    plot_feature_importance(
        rf_model, X.columns,
        f'Feature Importance - {ibd_type.upper()}',
        f'../docs/figures/{ibd_type}_feature_importance.png'
    )
    
    # Learning Curves
    plot_learning_curves(
        rf_model, X_train_res, y_train_res,
        f'Learning Curves - {ibd_type.upper()}',
        f'../docs/figures/{ibd_type}_learning_curves.png'
    )
    
    # ROC Curves
    plot_roc_curves(
        y_test, y_pred_proba,
        f'ROC Curves (One-vs-Rest) - {ibd_type.upper()}',
        f'../docs/figures/{ibd_type}_roc_curves.png'
    )
    
    # 5. SAVE MODEL
    model_path = f'../models/{ibd_type}/rf_severity_classifier_global.pkl'
    with open(model_path, 'wb') as f:
        pickle.dump(rf_model, f)
    print(f"\n💾 Modelo guardado: {model_path}")
    
    # Metadata
    metadata = {
        'ibd_type': ibd_type,
        'n_samples': len(df),
        'n_features': len(X.columns),
        'features': list(X.columns),
        'cv_accuracy_mean': float(cv_scores.mean()),
        'cv_accuracy_std': float(cv_scores.std()),
        'test_accuracy': float(test_acc),
        'test_f1_weighted': float(test_f1),
        'gap_cv_test': float(abs(cv_scores.mean() - test_acc)),
        'data_leakage_fixed': True
    }
    
    with open(f'../models/{ibd_type}/model_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"✅ {ibd_type.upper()} completado!\n")
    return rf_model, metadata

print("✓ Función de entrenamiento definida")

## 🔄 Entrenar Crohn

In [ ]:
rf_crohn, crohn_meta = train_with_cv('crohn')

## 🔄 Entrenar Colitis Ulcerosa

In [ ]:
rf_cu, cu_meta = train_with_cv('cu')

## 📊 Comparación Final: Antes vs Después

In [ ]:
print("\n" + "="*80)
print("COMPARACIÓN: ANTES vs DESPUÉS (Data Leakage Fix)")
print("="*80)

comparison = pd.DataFrame({
    'Metric': ['Accuracy', 'Features', 'Data Leakage', 'CV Gap', 'Interpretabilidad'],
    'ANTES (con leakage)': ['99.5%', '34', 'SÍ ⚠️', 'N/A', 'Baja'],
    'DESPUÉS (sin leakage)': [
        f"{crohn_meta['test_accuracy']:.1%}",
        str(crohn_meta['n_features']),
        'NO ✅',
        f"{crohn_meta['gap_cv_test']:.3f}",
        'Alta'
    ]
})

print("\n📊 CROHN:")
print(comparison.to_string(index=False))

print(f"\n✅ Accuracy realista: {crohn_meta['test_accuracy']:.1%}")
print(f"✅ CV Score: {crohn_meta['cv_accuracy_mean']:.3f} (+/- {crohn_meta['cv_accuracy_std']:.3f})")
print(f"✅ Gap CV-Test: {crohn_meta['gap_cv_test']:.3f}")

if crohn_meta['test_accuracy'] < 0.95:
    print("\n🎯 ÉXITO: Accuracy < 95% indica modelo realista sin data leakage!")
else:
    print("\n⚠️  WARNING: Accuracy todavía muy alta, revisar features")

print("\n📂 Gráficos generados:")
print("  - ../docs/figures/crohn_confusion_matrix.png")
print("  - ../docs/figures/crohn_feature_importance.png")
print("  - ../docs/figures/crohn_learning_curves.png")
print("  - ../docs/figures/crohn_roc_curves.png")

print("\n" + "="*80)
print("✅ ENTRENAMIENTO COMPLETADO - MODELOS VALIDADOS")
print("="*80)